In [6]:
import os
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import librosa

# Function to load a single audio file and extract features
def extract_features(file_path, duration=3):
  try:
    print(f"Processing file: {file_path}")  # Added for tracking
    y, sr = librosa.load(file_path, sr=None, duration=duration)
    # ... rest of feature extraction code (e.g., MFCCs)
    return mfccs
  except (IOError, EOFError) as e:
    print(f"Error encountered (likely file format issue) while parsing file: {file_path}")
    return None
  except Exception as e:
    print(f"Unexpected error encountered while parsing file: {file_path}")
    return None

# Function to load GTZAN dataset
def load_gtzan_dataset(dataset_path):
  songs = []
  genres = []
  for root, dirs, files in os.walk(dataset_path):
    for file in files:
      if file.endswith(".wav"):  # Assuming WAV format
        file_path = os.path.join(root, file)
        print(f"Loading file: {file_path}")  # Added for tracking
        genre = os.path.basename(root)
        features = extract_features(file_path)
        if features is not None:
          songs.append(features)
          genres.append(genre)
        else:
          print(f"Failed to extract features from file: {file_path}")
  return songs, genres


# Step 1: Data preprocessing
def prepare_sequences(songs, genres, duration=3):
  sequence_length = 100  # Adjust as needed
  network_input = []
  network_output = []
  for i, (song, genre) in enumerate(zip(songs, genres)):
    print(f"Processing file: {song}")  # Added for tracking
    try:
      # Load only the first 3 seconds of each song
      song_features = extract_features(song, duration=duration)
      if song_features is not None and song_features.shape[1] > sequence_length:
        for j in range(0, song_features.shape[1] - sequence_length, 1):
          sequence_in = song_features[:, j:j + sequence_length]
          network_input.append(sequence_in)
          network_output.append(genre)  # Append genre label for each sequence
    except Exception as e:
      print(f"Error processing file: {song}, Error: {e}")
  network_input = np.array(network_input)
  # Check if network_input is empty (commented out for now)
  # if network_input.size == 0:
  #   raise ValueError("No valid features extracted from the dataset.")
  # Normalize input (commented out for testing purposes)
  # network_input = network_input / np.amax(np.abs(network_input))
  return network_input, network_output

# Step 2: Define the LSTM model
def create_network(network_input, n_vocab):
  model = Sequential()
  model.add(LSTM(
      512,
      input_shape=(network_input.shape[1], network_input.shape[2]),
      return_sequences=True
  ))
  model.add(Dropout(0.3))
  model.add(LSTM(512, return_sequences=True))
  model.add(Dropout(0.3))
  model.add(LSTM(512))
  model.add(Dense(256))
  model.add(Dropout(0.3))
  model.add(Dense(n_vocab))
  model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
  return model

# Step 3: Train the LSTM model
def train(model, network_input, network_output):
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
      filepath,
      monitor='loss',
      verbose=0,
      save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]
    model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)
    
    # Step 4: Generate music
def generate_notes(model, network_input, pitchnames):
    start = np.random.randint(0, len(network_input)-1)
    pattern = network_input[start]
    prediction_output = []
    for note_index in range(10):  # Adjust as needed
        prediction_input = np.reshape(pattern, (1, pattern.shape[0], pattern.shape[1]))
        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = pitchnames[index]
        prediction_output.append(result)
        pattern = np.concatenate((pattern[:, 1:, :], prediction_input[:, :, -1:]), axis=1)
    return prediction_output

def main():
    dataset_path = "C:/Users/AsyrafAmeran/Desktop/AMG/AI-Music-Generation/Data/genres_original"  
    songs, genres = load_gtzan_dataset(dataset_path)
    network_input, network_output = prepare_sequences(songs, genres, duration=3)
    # Example: convert genre labels to categorical for training
    genre_to_int = {genre: i for i, genre in enumerate(set(genres))}
    network_output = np_utils.to_categorical([genre_to_int[genre] for genre in genres])
    
    model = create_network(network_input, len(set(genres)))
    train(model, network_input, network_output)
    prediction_output = generate_notes(model, network_input, set(genres))

    # Save the trained model
    model.save('music_generation_model.h5')

if __name__ == '__main__':
    main()


Loading file: C:/Users/AsyrafAmeran/Desktop/AMG/AI-Music-Generation/Data/genres_original\blues\blues.00000.wav
Processing file: C:/Users/AsyrafAmeran/Desktop/AMG/AI-Music-Generation/Data/genres_original\blues\blues.00000.wav
Unexpected error encountered while parsing file: C:/Users/AsyrafAmeran/Desktop/AMG/AI-Music-Generation/Data/genres_original\blues\blues.00000.wav
Failed to extract features from file: C:/Users/AsyrafAmeran/Desktop/AMG/AI-Music-Generation/Data/genres_original\blues\blues.00000.wav
Loading file: C:/Users/AsyrafAmeran/Desktop/AMG/AI-Music-Generation/Data/genres_original\blues\blues.00001.wav
Processing file: C:/Users/AsyrafAmeran/Desktop/AMG/AI-Music-Generation/Data/genres_original\blues\blues.00001.wav
Unexpected error encountered while parsing file: C:/Users/AsyrafAmeran/Desktop/AMG/AI-Music-Generation/Data/genres_original\blues\blues.00001.wav
Failed to extract features from file: C:/Users/AsyrafAmeran/Desktop/AMG/AI-Music-Generation/Data/genres_original\blues\blue

ValueError: zero-size array to reduction operation maximum which has no identity